# HURDAT2 Analysis: Storms Near Florida (2004-2025)

This notebook downloads and analyzes hurricane data from the HURDAT2 database, focusing on storms that came within 60 nautical miles of Florida from 2004 to 2025.

## 1. Import Libraries and Configure Paths

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, box
from shapely import wkt
import urllib.request
import tempfile
from pathlib import Path
from math import radians, cos, sin, asin, sqrt
import warnings
warnings.filterwarnings('ignore')

# Setup paths
BASE_PATH = Path.cwd().parent if 'code' in Path.cwd().name else Path.cwd()
DATA_RAW = BASE_PATH / "data" / "raw"
DATA_PROCESSED = BASE_PATH / "data" / "processed"
DATA_FINAL = BASE_PATH / "data" / "final"

# Ensure directories exist
for path in [DATA_RAW, DATA_PROCESSED, DATA_FINAL]:
    path.mkdir(parents=True, exist_ok=True)

print(f"Base path: {BASE_PATH}")
print(f"Data raw: {DATA_RAW}")
print(f"Data processed: {DATA_PROCESSED}")

## 2. Load HURDAT2 Data (2004–2025)

In [ ]:
def download_hurdat2():
    """Download HURDAT2 data from NOAA if not already cached."""
    url = "https://www.nhc.noaa.gov/data/hurdat2/hurdat2.txt"
    output_file = DATA_RAW / "hurdat2_raw.txt"
    
    if output_file.exists():
        print(f"HURDAT2 data already exists at {output_file}")
        return output_file
    
    print(f"Downloading HURDAT2 data from {url}...")
    try:
        urllib.request.urlretrieve(url, output_file)
        print(f"✓ Successfully saved to {output_file}")
        return output_file
    except Exception as e:
        print(f"Error downloading: {e}")
        return None

def parse_hurdat2(file_path):
    """
    Parse HURDAT2 text file into a DataFrame.
    Header lines contain storm info, data lines contain position/intensity.
    """
    records = []
    current_storm = None
    
    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            
            # Header line (contains alphabetic character at start)
            if line[0].isalpha():
                parts = [p.strip() for p in line.split(',')]
                if len(parts) >= 2:
                    current_storm = {
                        'id': parts[0],
                        'name': parts[1],
                        'num_records': int(parts[2])
                    }
            else:
                # Data line (track point)
                if current_storm:
                    parts = [p.strip() for p in line.split(',')]
                    if len(parts) >= 8:
                        try:
                            # Parse latitude (format: DDNH or DDDS)
                            lat_str = parts[4]
                            lat_val = float(lat_str[:-1])
                            lat = lat_val if lat_str[-1] == 'N' else -lat_val
                            
                            # Parse longitude (format: DDDE or DDDW)
                            lon_str = parts[5]
                            lon_val = float(lon_str[:-1])
                            lon = lon_val if lon_str[-1] == 'E' else -lon_val
                            
                            record = {
                                'storm_id': current_storm['id'],
                                'storm_name': current_storm['name'],
                                'date': parts[0],
                                'time': parts[1],
                                'record_id': parts[2],
                                'status': parts[3],
                                'lat': lat,
                                'lon': lon,
                                'max_wind': int(parts[6]) if parts[6].lstrip('-').isdigit() else np.nan,
                                'min_pressure': int(parts[7]) if parts[7].lstrip('-').isdigit() else np.nan,
                            }
                            records.append(record)
                        except (ValueError, IndexError):
                            continue
    
    df = pd.DataFrame(records)
    if not df.empty:
        df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')
        df['year'] = df['date'].dt.year
    
    return df

# Download and parse data
hurdat_file = download_hurdat2()
if hurdat_file:
    print("\nParsing HURDAT2 data...")
    hurdat_df = parse_hurdat2(hurdat_file)
    print(f"✓ Total records: {len(hurdat_df):,}")
    print(f"✓ Date range: {hurdat_df['date'].min()} to {hurdat_df['date'].max()}")
    print(f"✓ Unique storms: {hurdat_df['storm_id'].nunique()}")
    print(f"\nFirst few records:")
    print(hurdat_df.head())
else:
    print("Failed to download HURDAT2 data")

## 3. Load Florida Boundary and Create GeoDataFrame

In [ ]:
try:
    import geopandas as gpd
    
    # Load US states boundary from Natural Earth data (includes Florida)
    # This uses the online source, which may take a moment
    print("Loading US states boundary data...")
    us_states = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
    us_states = us_states[us_states['iso_a3'] == 'USA']
    
    # For more detailed Florida boundary, try Natural Earth data
    # If that fails, we'll create a bounding box approximation
    try:
        # Attempt to load higher resolution state boundaries
        states = gpd.read_file('https://naciscdn.org/naturalearth/10m/admin_1_states_provinces.zip')
        florida = states[(states['admin'] == 'United States of America') & 
                        (states['name'] == 'Florida')]
        if not florida.empty:
            print("✓ Loaded detailed Florida boundary from Natural Earth")
        else:
            raise Exception("Florida not found in Natural Earth data")
    except:
        # Fallback: Create Florida bounding polygon based on approximate coordinates
        # Florida approximately spans: 24.5°N to 30.5°N, 80°W to 87.5°W
        print("Creating Florida boundary polygon (approximate)...")
        florida_bounds = box(ymin=24.5, ymax=30.5, xmin=-87.5, xmax=-80.0)
        florida = gpd.GeoDataFrame({'name': ['Florida (approximate)'], 'geometry': [florida_bounds]})
    
    florida = florida.to_crs(epsg=3857)  # Web Mercator projection
    print(f"✓ Florida boundary loaded")
    print(f"  CRS: {florida.crs}")
    print(f"  Bounds: {florida.total_bounds}")
    
except ImportError:
    print("geopandas or dependencies not installed. Installing...")
    import subprocess
    subprocess.check_call(['pip', 'install', 'geopandas', '-q'])
    import geopandas as gpd
    
    # Try again
    print("Loading US states boundary data...")
    us_states = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
    florida_bounds = box(ymin=24.5, ymax=30.5, xmin=-87.5, xmax=-80.0)
    florida = gpd.GeoDataFrame({'name': ['Florida (approximate)'], 'geometry': [florida_bounds]})
    florida = florida.to_crs(epsg=3857)
    print("✓ Florida boundary loaded (approximate)")

## 4. Convert Storm Points to GeoDataFrame and Filter by Year

In [ ]:
# Filter to 2004-2025 period first
hurdat_2004_2025 = hurdat_df[(hurdat_df['year'] >= 2004) & (hurdat_df['year'] <= 2025)].copy()

print(f"Records in 2004-2025 period: {len(hurdat_2004_2025):,}")
print(f"Storms in 2004-2025 period: {hurdat_2004_2025['storm_id'].nunique()}")
print(f"Date range: {hurdat_2004_2025['date'].min()} to {hurdat_2004_2025['date'].max()}")

# Convert to GeoDataFrame with point geometries
geometry = [Point(xy) for xy in zip(hurdat_2004_2025['lon'], hurdat_2004_2025['lat'])]
hurdat_gdf = gpd.GeoDataFrame(hurdat_2004_2025, geometry=geometry, crs='EPSG:4326')

# Convert to same projection as Florida for distance calculations
hurdat_gdf = hurdat_gdf.to_crs(epsg=3857)

print(f"\n✓ Created GeoDataFrame with {len(hurdat_gdf)} points")
print(f"  CRS: {hurdat_gdf.crs}")
print(f"\nSample of converted data:")
print(hurdat_gdf[['date', 'storm_name', 'lat', 'lon', 'max_wind', 'geometry']].head())

## 5. Filter Points Within 60 Nautical Miles of Florida

In [ ]:
# Convert 60 nautical miles to meters
# 1 nautical mile = 1852 meters
BUFFER_DISTANCE_NM = 60
BUFFER_DISTANCE_M = BUFFER_DISTANCE_NM * 1852

print(f"Creating {BUFFER_DISTANCE_NM} nautical mile buffer around Florida...")
print(f"  {BUFFER_DISTANCE_NM} NM = {BUFFER_DISTANCE_M:,} meters")

# Create buffer around Florida
florida_buffer = florida.copy()
florida_buffer['geometry'] = florida_buffer.geometry.buffer(BUFFER_DISTANCE_M)

# Find all hurricanes that intersect the buffered area
hurdat_near_florida = hurdat_gdf[hurdat_gdf.geometry.within(florida_buffer.geometry.iloc[0])].copy()

print(f"\n✓ Storms coming within {BUFFER_DISTANCE_NM} NM of Florida (2004-2025):")
print(f"  Total points: {len(hurdat_near_florida):,}")
print(f"  Unique storms: {hurdat_near_florida['storm_id'].nunique()}")
print(f"  Year range: {hurdat_near_florida['year'].min()} to {hurdat_near_florida['year'].max()}")

# Alternative: Calculate minimum distance for each storm
print("\nCalculating closest approach to Florida for each storm...")
def min_distance_from_florida(group):
    distances = group.geometry.distance(florida.geometry.iloc[0])
    return distances.min() / 1852  # Convert back to nautical miles

storm_distances = hurdat_gdf.groupby('storm_id').apply(min_distance_from_florida)
storms_near_florida = storm_distances[storm_distances <= BUFFER_DISTANCE_NM].index.tolist()

print(f"Storms with closest point within {BUFFER_DISTANCE_NM} NM: {len(storms_near_florida)}")

# Get the detailed records for these storms
hurdat_near_florida_alt = hurdat_gdf[hurdat_gdf['storm_id'].isin(storms_near_florida)].copy()

print(f"\nDetailed records for {BUFFER_DISTANCE_NM} NM storms: {len(hurdat_near_florida_alt):,}")
print(f"Unique storms: {hurdat_near_florida_alt['storm_id'].nunique()}")

## 6. Summarize and Visualize Nearby Storm Activity

In [ ]:
# Create summary table
print("="*80)
print(f"HURRICANES/TROPICAL STORMS WITHIN {BUFFER_DISTANCE_NM} NM OF FLORIDA")
print(f"Years 2004-2025")
print("="*80)

# Get summary for each storm
storm_summary_list = []
for storm_id in storms_near_florida:
    storm_data = hurdat_near_florida_alt[hurdat_near_florida_alt['storm_id'] == storm_id]
    storm_name = storm_data['storm_name'].iloc[0]
    year = storm_data['year'].iloc[0]
    
    # Find minimum distance
    min_dist_m = storm_data.geometry.distance(florida.geometry.iloc[0]).min()
    min_dist_nm = min_dist_m / 1852
    
    # Get peak intensity
    max_wind = storm_data['max_wind'].max()
    min_pressure = storm_data['min_pressure'].min()
    
    # Get date range
    start_date = storm_data['date'].min()
    end_date = storm_data['date'].max()
    
    storm_summary_list.append({
        'Year': year,
        'Storm ID': storm_id,
        'Storm Name': storm_name,
        'Closest Distance (NM)': round(min_dist_nm, 1),
        'Max Wind Speed (kt)': int(max_wind) if pd.notna(max_wind) else 'N/A',
        'Min Pressure (mb)': int(min_pressure) if pd.notna(min_pressure) else 'N/A',
        'Start Date': start_date.strftime('%Y-%m-%d'),
        'End Date': end_date.strftime('%Y-%m-%d')
    })

summary_df = pd.DataFrame(storm_summary_list).sort_values('Year')

print("\n")
print(summary_df.to_string(index=False))

# Save to CSV
summary_csv = DATA_PROCESSED / f"florida_storms_{BUFFER_DISTANCE_NM}nm_2004_2025.csv"
summary_df.to_csv(summary_csv, index=False)
print(f"\n✓ Summary saved to: {summary_csv}")

# Save detailed GeoJSON
geojson_file = DATA_PROCESSED / f"florida_storms_{BUFFER_DISTANCE_NM}nm_2004_2025.geojson"
hurdat_near_florida_alt[['date', 'storm_id', 'storm_name', 'year', 'lat', 'lon', 'max_wind', 'min_pressure', 'geometry']].to_file(geojson_file, driver='GeoJSON')
print(f"✓ GeoJSON saved to: {geojson_file}")

# Summary statistics by year
print("\n" + "="*80)
print("STORM COUNT BY YEAR")
print("="*80)
yearly_summary = summary_df.groupby('Year').size()
print(yearly_summary)

print(f"\nTotal storms affecting Florida (2004-2025): {len(summary_df)}")
print(f"Average storms per year: {len(summary_df) / 22:.1f}")

## 7. Visualizations

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.patches import Circle

# Create a map visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))

# Plot 1: Storm tracks and Florida boundary with buffer
ax1.set_title(f'Hurricane Tracks within {BUFFER_DISTANCE_NM} NM of Florida\n(2004-2025)', fontsize=14, fontweight='bold')

# Plot Florida (convert back to lat/lon for visualization)
florida_plot = florida.to_crs('EPSG:4326')
florida_plot.plot(ax=ax1, alpha=0.3, edgecolor='green', facecolor='lightgreen', label='Florida')

# Plot 60 NM buffer (approximate circle on map)
florida_center_lon = florida_plot.geometry.centroid.x.iloc[0]
florida_center_lat = florida_plot.geometry.centroid.y.iloc[0]
# Approximate circle (in degrees, rough conversion)
buffer_deg = BUFFER_DISTANCE_NM / 69  # 1 degree ≈ 69 miles / ≈ 59 NM
circle = Circle((florida_center_lon, florida_center_lat), buffer_deg, 
                color='blue', fill=False, linestyle='--', linewidth=2, label=f'{BUFFER_DISTANCE_NM} NM buffer')
ax1.add_patch(circle)

# Plot hurricane points and color by wind speed
hurdat_plot = hurdat_near_florida_alt.to_crs('EPSG:4326')
scatter = ax1.scatter(hurdat_plot.geometry.x, hurdat_plot.geometry.y, 
                     c=hurdat_plot['max_wind'], cmap='YlOrRd', s=20, alpha=0.6, 
                     vmin=35, vmax=150)

ax1.set_xlabel('Longitude', fontsize=11)
ax1.set_ylabel('Latitude', fontsize=11)
ax1.legend(loc='upper left')
ax1.grid(True, alpha=0.3)
cbar1 = plt.colorbar(scatter, ax=ax1)
cbar1.set_label('Wind Speed (knots)', fontsize=10)

# Plot 2: Storm count by year
yearly_count = summary_df.groupby('Year').size()
ax2.bar(yearly_count.index, yearly_count.values, color='steelblue', edgecolor='black', alpha=0.7)
ax2.set_title('Tropical Cyclones Affecting Florida Area\nby Year (2004-2025)', fontsize=14, fontweight='bold')
ax2.set_xlabel('Year', fontsize=11)
ax2.set_ylabel('Number of Storms', fontsize=11)
ax2.grid(True, alpha=0.3, axis='y')
ax2.set_xlim(2003.5, 2025.5)

# Add average line
avg_storms = len(summary_df) / 22
ax2.axhline(y=avg_storms, color='red', linestyle='--', linewidth=2, label=f'Average: {avg_storms:.1f}')
ax2.legend()

plt.tight_layout()
fig_path = Path(__file__).parent.parent / "results" / "figures" / f"florida_storms_{BUFFER_DISTANCE_NM}nm_analysis.png"
fig_path.parent.mkdir(parents=True, exist_ok=True)
plt.savefig(fig_path, dpi=300, bbox_inches='tight')
print(f"✓ Map saved to: {fig_path}")
plt.show()

print("\nVisualization complete!")